## Imports

In [ ]:
!pip install datasets

In [2]:
from datasets import load_dataset
from collections import Counter
from src.preprocess import preprocess
from src.error_injection import apply_errors_to_segment, error_functions

## Load the Dataset

In [3]:
train_ds = load_dataset("Abdou/arabic-tashkeel-dataset", split="train")
#valid_ds = load_dataset("Abdou/arabic-tashkeel-dataset", split="validation")
#test_ds = load_dataset("Abdou/arabic-tashkeel-dataset", split="test")

## EDA

In [4]:
# see the dataset columns
print(f'Columns: {train_ds.column_names}')

# remove the "source" column
train_ds = train_ds.to_pandas().drop('source', axis=1)

# see the dataset columns again
print(f'\nColumns: {train_ds.columns}')


Columns: ['non_vocalized', 'vocalized', 'source']

Columns: Index(['non_vocalized', 'vocalized'], dtype='object')


In [5]:
# show a row

print(':بدون تشكيل')
print(train_ds.loc[0, 'non_vocalized'])

print('\n:بتشكيل')
print(train_ds.loc[0, 'vocalized'])

:بدون تشكيل
حدثنا محمد بن أحمد أبو عبد الله بمكة، حدثنا أحمد بن رشدين ⦗٧٢⦘، حدثني أبي، عن أبيه، عن ابن لهيعة، عن عقيل، عن ابن شهاب، عن أنس بن مالك، عن النبي صلى الله عليه وسلم أنه دخل مكة عام الفتح وعلى رأسه المغفر، فلما نزعه جاءه رجل، فقال: ابن خطل متعلق بأستار الكعبة، فقال رسول الله صلى الله عليه وسلم: «اقتلوه» قال ابن شهاب ولم يكن رسول الله صلى الله عليه وسلم يومئذ محرما

:بتشكيل
حَدَّثَنَا مُحَمَّدُ بْنُ أَحْمَدَ أَبُو عَبْدِ اللَّهِ بِمَكَّةَ، حَدَّثَنَا أَحْمَدُ بْنُ رِشْدِينَ ⦗٧٢⦘، حَدَّثَنِي أَبِي، عَنْ أَبِيهِ، عَنِ ابْنِ لَهِيعَةَ، عَنْ عَقِيلٍ، عَنِ ابْنِ شِهَابٍ، عَنْ أَنَسِ بْنِ مَالِكٍ، عَنِ النَّبِيِّ صَلَّى اللهُ عَلَيْهِ وَسَلَّمَ أَنَّهُ دَخَلَ مَكَّةَ عَامَ الْفَتْحِ وَعَلَى رَأْسِهِ الْمِغْفَرُ، فَلَمَّا نَزَعَهُ جَاءَهُ رَجُلٌ، فَقَالَ: ابْنُ خَطَلٍ مُتَعَلِّقٌ بِأَسْتَارِ الْكَعْبَةِ، فَقَالَ رَسُولُ اللَّهِ صَلَّى اللهُ عَلَيْهِ وَسَلَّمَ: «اقْتُلُوهُ» قَالَ ابْنُ شِهَابٍ وَلَمْ يَكُنْ رَسُولُ اللَّهِ صَلَّى اللهُ عَلَيْهِ وَسَلَّمَ يَوْمَئِذٍ مُحْرِمًا


In [7]:
# # see the set of characters the dataset is composed of and their counts

# char_counts = Counter()

# for row in train_ds.itertuples():
#     char_counts.update(row.vocalized)

# unique_chars = set(char_counts.keys())

# print("Unique characters:", unique_chars)
# print("\nCharacter frequencies:")
# for char, count in char_counts.items():
#     print(f"'{char}': {count}")


Unique characters: {'⟶', '言', '𒆍', 'ἡ', 'ス', '豹', 'Í', 'ي', 'ő', 'ા', '<', '\u202b', '泉', 'բ', '備', 'λ', '纳', '洲', 'ὶ', 'Ø', 'ῦ', '𐎛', '阳', '邦', '𠮉', '̅', 'ベ', 'ϕ', '𐀮', '戰', '会', '\u200f', '半', '게', 'ь', '양', '衣', '王', '髙', '堂', '٭', '斐', 'ੰ', 'ⵖ', 'छ', '千', '秦', '熊', 'ㅟ', '署', '♯', '印', '⁰', 'І', 'ṣ', '倒', '祖', '順', '浜', '်', 'ײ', '¤', 'ຊ', 'ἒ', 'ダ', '将', '东', 'ད', 'ঙ', '彰', 'Ż', '云', '崇', '#', '―', '송', '祐', '𐭥', '̀', 'ु', 'Ā', '紙', '清', 'μ', '🙏', 'Э', 'ⴽ', '正', '😉', '剣', 'ދ', 'ş', 'ٔ', '蔵', '宾', '遊', 'カ', 'Ф', 'ត', '해', '𐤍', '業', 'ח', '件', 'ዝ', '事', 'ٜ', '鬼', '워', 'ⵕ', 'п', '☆', 'コ', '男', 'ހ', 'ラ', '۱', 'ລ', 'τ', '˨', 'c', '訪', '⁻', '區', 'ۛ', 'ম', 'ῥ', '那', '𐭮', 'ٞ', 'ু', 'ُ', 'Ч', '德', 'ⴲ', 'ማ', '儒', 'İ', 'ω', 'ջ', '町', '4', '郷', '交', '≤', '忽', 'ο', '牙', '走', 'ਮ', '낙', '俗', '造', 'ً', '𠮈', '童', 'ێ', '虐', 'ㄱ', '利', '解', 'Ş', '།', '祭', '雄', 'ⵎ', '⁹', 'イ', '塘', 'ਯ', 'क', 'Р', 'ⁿ', '气', '明', 'も', 'ध', '٥', '建', 'ⲡ', '杉', 'σ', '日', '안', 'ۨ', '武', 'X', 'や', 'ぼ', '𠮡', '虎', '蕃', '石', 'E', 

## Preprocessing

In [5]:
train_ds['vocalized_filtered'] = train_ds['vocalized'].apply(preprocess)

# check the result
print(train_ds.loc[0, 'vocalized'])
print(train_ds.loc[0, 'vocalized_filtered'])

حَدَّثَنَا مُحَمَّدُ بْنُ أَحْمَدَ أَبُو عَبْدِ اللَّهِ بِمَكَّةَ، حَدَّثَنَا أَحْمَدُ بْنُ رِشْدِينَ ⦗٧٢⦘، حَدَّثَنِي أَبِي، عَنْ أَبِيهِ، عَنِ ابْنِ لَهِيعَةَ، عَنْ عَقِيلٍ، عَنِ ابْنِ شِهَابٍ، عَنْ أَنَسِ بْنِ مَالِكٍ، عَنِ النَّبِيِّ صَلَّى اللهُ عَلَيْهِ وَسَلَّمَ أَنَّهُ دَخَلَ مَكَّةَ عَامَ الْفَتْحِ وَعَلَى رَأْسِهِ الْمِغْفَرُ، فَلَمَّا نَزَعَهُ جَاءَهُ رَجُلٌ، فَقَالَ: ابْنُ خَطَلٍ مُتَعَلِّقٌ بِأَسْتَارِ الْكَعْبَةِ، فَقَالَ رَسُولُ اللَّهِ صَلَّى اللهُ عَلَيْهِ وَسَلَّمَ: «اقْتُلُوهُ» قَالَ ابْنُ شِهَابٍ وَلَمْ يَكُنْ رَسُولُ اللَّهِ صَلَّى اللهُ عَلَيْهِ وَسَلَّمَ يَوْمَئِذٍ مُحْرِمًا
حَدَّثَنَا مُحَمَّدُ بْنُ أَحْمَدَ أَبُو عَبْدِ اللَّهِ بِمَكَّةَ، حَدَّثَنَا أَحْمَدُ بْنُ رِشْدِينَ ، حَدَّثَنِي أَبِي، عَنْ أَبِيهِ، عَنِ ابْنِ لَهِيعَةَ، عَنْ عَقِيلٍ، عَنِ ابْنِ شِهَابٍ، عَنْ أَنَسِ بْنِ مَالِكٍ، عَنِ النَّبِيِّ صَلَّى اللهُ عَلَيْهِ وَسَلَّمَ أَنَّهُ دَخَلَ مَكَّةَ عَامَ الْفَتْحِ وَعَلَى رَأْسِهِ الْمِغْفَرُ، فَلَمَّا نَزَعَهُ جَاءَهُ رَجُلٌ، فَقَالَ: ابْنُ خَطَلٍ مُتَ

## Error Injection

In [ ]:
# train_ds['vocalized_errors'] = train_ds['vocalized_filtered'].apply(
#     lambda x: apply_errors_to_segment(x, error_functions, error_probability=0.1)
# )

In [6]:
# apply on the first row only

print(train_ds.loc[0, 'vocalized'])
print(train_ds.loc[0, 'vocalized_filtered'])

# apply errors on the first row only
print(apply_errors_to_segment(train_ds.loc[0, 'vocalized_filtered'], error_functions, error_probability=0.1))

حَدَّثَنَا مُحَمَّدُ بْنُ أَحْمَدَ أَبُو عَبْدِ اللَّهِ بِمَكَّةَ، حَدَّثَنَا أَحْمَدُ بْنُ رِشْدِينَ ⦗٧٢⦘، حَدَّثَنِي أَبِي، عَنْ أَبِيهِ، عَنِ ابْنِ لَهِيعَةَ، عَنْ عَقِيلٍ، عَنِ ابْنِ شِهَابٍ، عَنْ أَنَسِ بْنِ مَالِكٍ، عَنِ النَّبِيِّ صَلَّى اللهُ عَلَيْهِ وَسَلَّمَ أَنَّهُ دَخَلَ مَكَّةَ عَامَ الْفَتْحِ وَعَلَى رَأْسِهِ الْمِغْفَرُ، فَلَمَّا نَزَعَهُ جَاءَهُ رَجُلٌ، فَقَالَ: ابْنُ خَطَلٍ مُتَعَلِّقٌ بِأَسْتَارِ الْكَعْبَةِ، فَقَالَ رَسُولُ اللَّهِ صَلَّى اللهُ عَلَيْهِ وَسَلَّمَ: «اقْتُلُوهُ» قَالَ ابْنُ شِهَابٍ وَلَمْ يَكُنْ رَسُولُ اللَّهِ صَلَّى اللهُ عَلَيْهِ وَسَلَّمَ يَوْمَئِذٍ مُحْرِمًا
حَدَّثَنَا مُحَمَّدُ بْنُ أَحْمَدَ أَبُو عَبْدِ اللَّهِ بِمَكَّةَ، حَدَّثَنَا أَحْمَدُ بْنُ رِشْدِينَ ، حَدَّثَنِي أَبِي، عَنْ أَبِيهِ، عَنِ ابْنِ لَهِيعَةَ، عَنْ عَقِيلٍ، عَنِ ابْنِ شِهَابٍ، عَنْ أَنَسِ بْنِ مَالِكٍ، عَنِ النَّبِيِّ صَلَّى اللهُ عَلَيْهِ وَسَلَّمَ أَنَّهُ دَخَلَ مَكَّةَ عَامَ الْفَتْحِ وَعَلَى رَأْسِهِ الْمِغْفَرُ، فَلَمَّا نَزَعَهُ جَاءَهُ رَجُلٌ، فَقَالَ: ابْنُ خَطَلٍ مُتَ